In [ ]:
# if (!require("BiocManager", quietly = TRUE))
#           install.packages("BiocManager")

# BiocManager::install("methylclock", force = TRUE)
# BiocManager::install("impute")

# install.package(tibble)
# install.package(ggplot2)
# install.package(ggpmisc)
# install.package(GEOquery)

# library(methylclockData)
# library(methylclock)
# library(Biobase)
# library(tibble)
# library(impute)
# library(ggplot2)
# library(ggpmisc)
# library(GEOquery)
# library(data.table)

# MethylationData <- fread('./two-way_normalized_beta_value_all_CpGs.txt', header=TRUE, sep = '\t')
# MethylationData <- data.frame(norm_by_bmiq_with_name)
# # MethylationData <- read.csv('/disk0/sm/methyl/03_pollution/total/two-way_normalized_beta_value_all_CpGs.txt', header=TRUE, sep = '\t')
# MethylationData$cpgs <- sub("_.+$", "", MethylationData$cpgs)
# head(MethylationData$cpgs)

# checkClocksGA(MethylationData)

# age.example55 <- DNAmAge(MethylationData)
# age.example55
# write.table(age.example55, file='./DNAmAge_Result.txt', sep='\t', row.names = F)

# age.example55.2 <- DNAmGA(MethylationData)
# age.example55.2
# write.table(age.example55.2, file='./DNAmGA_Result.txt', sep='\t', row.names = F)

In [1]:
library(ggplot2)
library(reshape)
library(RColorBrewer)
library(stringr)
library(dplyr)
library(reshape2)
library(grid)


Attaching package: ‘dplyr’


The following object is masked from ‘package:reshape’:

    rename


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘reshape2’


The following objects are masked from ‘package:reshape’:

    colsplit, melt, recast




In [2]:
library(doParallel)

registerDoParallel(cores=24)

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



In [3]:
age_normal <- read.table("/disk0/sm/methyl/03_pollution/total/data/age.txt", sep="\t", fill = TRUE, header=TRUE)
head(age_normal)

,id,Name,phase,Horvath,Hannum,Levine,skinHorvath,PedBE,Wu,BLUP,EN,SM
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,NIH23J4322396_11,NIH23J4322396,11기,55.51412,19.811989,50.54983,49.12250,13.85461,10.717376,56.28313,53.43683,58.26551
2,NIH23J4322396_1,NIH23J4322396,1기,41.99686,5.914913,39.59200,31.02157,12.88426,8.895243,37.79080,33.26235,39.65860
3,NIH23J4914208_11,NIH23J4914208,11기,67.91160,31.218789,60.87453,54.32142,16.24088,11.039546,63.49034,56.66958,61.51715
4,NIH23J4914208_1,NIH23J4914208,1기,54.59198,20.199773,40.52585,41.40881,15.09632,10.547272,49.85410,43.53373,46.07019
5,NIH23J4098396_11,NIH23J4098396,11기,74.42639,35.501763,55.03708,58.04022,15.30015,11.600250,64.06980,60.66288,65.73191
6,NIH23J4098396_1,NIH23J4098396,1기,54.45880,19.403784,40.35036,39.18995,13.25277,10.084851,44.87467,41.59730,46.12018


In [4]:
axis_num <- 'black'

# x=reorder(Name, Horvath)
p <- ggplot(data=age_normal, aes(x=reorder(Name, Horvath), y=Horvath, group=phase, color=phase)) + #reorder(id, value)
geom_smooth(size=0.5, level=0.999) +#, color=adjustcolor('red', alpha.f =0.5)) +
geom_point(size=0.5) +
coord_fixed(ratio = 0.5) +
  coord_cartesian(ylim = c(40, 110))  +
#   scale_x_continuous(breaks = seq(0, nrow(all_callrate), nrow(all_callrate)/plate_num)) +
  scale_y_continuous(breaks = seq(40, 110, 10)) +
  labs(title = "Horvath", 
        x = 'sample', 
        y = 'age difference') + 
  theme_bw() +
  theme(
  #   axis.text.x = element_text(seq(0, 2198), angle=30, hjust=1),
    axis.text.x = element_blank(),
    axis.text.y = element_text(size=18),
    plot.title = element_text(hjust = 0.5,size=20,face='bold'),
    axis.title.x = element_blank(),
    axis.title.y = element_blank(),
    axis.ticks.x = element_blank(),
    legend.text = element_text(size=18),
    legend.position = 'none',
    
    panel.grid.major.x = element_blank(), 
    panel.grid.major.y = element_line(linetype = "dotted", colour = "grey80"),
    panel.grid.minor.x = element_blank(), 
    panel.grid.minor.y = element_blank(), 
    legend.title = element_blank()
    ) +
  # geom_hline(yintercept=c(mean(age_1_11$Horvath), mean(age_1_11$Hannum), mean(age_1_11$Levine), mean(age_1_11$skinHorvath), mean(age_1_11$BLUP), mean(age_1_11$EN)), color=c('#FB8072', '#FDB462', '#FFED6F', '#B3DE69', '#80B1D3', '#BC80BD')) + 
  scale_colour_manual(values=c('#3A4D39', '#FF9209')) 
ggsave(p, file='/disk0/sm/methyl/03_pollution/total/plot/age_Horvath_check.png', width=6000, height=6000, units='px', dpi=1000)

Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”
Coordinate system already present. Adding new coordinate system, which will
replace the existing one.
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'


In [11]:
age <- read.table("/disk0/sm/methyl/03_pollution/total/data/age_1_11.txt", sep="\t", fill = TRUE, header=TRUE)

age <- age[order(age$Horvath), ]
# head(age)
age$id <- factor(age$Name, level=age$Name)

m_age = melt(age)
# head(m_age)
summary(age)

Using Name, id as id variables



     Name              Horvath            Hannum           Levine      
 Length:1053        Min.   : 0.8815   Min.   : 3.305   Min.   :-1.081  
 Class :character   1st Qu.:15.4061   1st Qu.:13.223   1st Qu.:14.678  
 Mode  :character   Median :18.0083   Median :15.066   Median :17.853  
                    Mean   :18.1112   Mean   :15.326   Mean   :18.043  
                    3rd Qu.:20.8065   3rd Qu.:17.245   3rd Qu.:21.073  
                    Max.   :34.1617   Max.   :30.990   Max.   :44.961  
                                                                       
  skinHorvath         PedBE              Wu               BLUP       
 Min.   : 3.348   Min.   :-2.632   Min.   :-4.4932   Min.   : 3.329  
 1st Qu.:16.146   1st Qu.: 2.313   1st Qu.: 0.9277   1st Qu.:16.722  
 Median :18.043   Median : 2.906   Median : 1.3631   Median :17.966  
 Mean   :17.837   Mean   : 2.976   Mean   : 1.3848   Mean   :17.898  
 3rd Qu.:19.589   3rd Qu.: 3.568   3rd Qu.: 1.8110   3rd Qu.:19.134  
 Max

In [10]:
colr =c('#FB8072', '#FDB462', '#FFED6F', '#B3DE69', '#80B1D3', '#BC80BD', '#ff6dbd', '#adadad')

In [7]:
m_age <- m_age %>% filter((variable == 'Horvath') | (variable == 'Levine') | (variable == 'BLUP') | (variable == 'SM'))

In [9]:
p <- ggplot(m_age, aes(x=variable, y=value, fill=variable)) + 
  # coord_cartesian(ylim = c(5, 35)) +
  geom_hline(yintercept=20, color='#E74646', size=0.7, linetype=2) +
  geom_boxplot(outlier.shape = NA) + #
  labs(
        x = 'age predict method', 
        y = 'age difference') + 
  # scale_fill_manual(values=c('#FB8072', '#FFED6F', '#ff6dbd', 'white')) +
  scale_fill_manual(values=colr) +
  theme_bw() +
  theme(
  #   axis.text.x = element_text(seq(0, 2198), angle=30, hjust=1),
    axis.text.x = element_text(size=15),
    axis.text.y = element_text(size=15),
    plot.title = element_text(hjust = 0.5,size=20,face='bold'),
    axis.title.x = element_blank(),
    axis.title.y = element_blank(),
    axis.ticks.x = element_blank(),
    legend.text = element_text(size=18),
    legend.position = 'none',
    
    panel.grid.major.x = element_line(linetype = "dotted", colour = "grey80"),
    panel.grid.major.y = element_line(linetype = "dotted", colour = "gray80"), 
    panel.grid.minor.x = element_blank(), 
    panel.grid.minor.y = element_blank(), 
    legend.title = element_blank()
    ) 
  # geom_jitter(shape=1, position=position_jitter(0.3), size=0.1) +
  #  geom_boxplot(width=0.1, outlier.shape = NA) 


  ggsave(p, file='/disk0/sm/methyl/03_pollution/total/plot/age_1_11_box.png', width=10000, height=10000, units='px', dpi=1000)
# dev.off()

In [27]:
# colr <- c('#FB8072', '#FDB462', '#FFED6F', '#B3DE69', '#80B1D3', '#BC80BD')

# p <- ggplot(m_age, aes(x=variable, y=value, fill=variable)) + 
#   coord_cartesian(ylim = c(5, 35)) +
#   geom_violin() + #
#   geom_boxplot(width=0.1, outlier.shape = NA) +
#   labs(title = "1 11", 
#         # x = 'algorithm', 
#         y = 'age difference') + 
#   scale_fill_manual(values=colr) +
#   theme_bw() +
#   theme(
#   #   axis.text.x = element_text(seq(0, 2198), angle=30, hjust=1),
#     axis.text.x = element_text(size=18),
#     axis.text.y = element_text(size=18),
#     plot.title = element_text(hjust = 0.5,size=20,face='bold'),
#     axis.title.x = element_blank(),
#     axis.title.y = element_blank(),
#     axis.ticks.x = element_blank(),
#     legend.text = element_text(size=18),
#     legend.position = 'none',
    
#     panel.grid.major.x = element_line(linetype = "dotted", colour = "grey80"),
#     panel.grid.major.y = element_line(linetype = "dotted", colour = "gray80"), 
#     panel.grid.minor.x = element_blank(), 
#     panel.grid.minor.y = element_blank(), 
#     legend.title = element_blank()
#     )
#   # geom_jitter(shape=1, position=position_jitter(0.3), size=0.1) +
#   #  geom_boxplot(width=0.1, outlier.shape = NA) 


#   ggsave(p, file='/disk0/sm/methyl/03_pollution/total/plot/age_1_11_violin.png', width=10000, height=10000, units='px', dpi=1000)
# # dev.off()

In [ ]:
age <- read.table("/disk0/sm/methyl/03_pollution/total/data/age.txt", sep="\t", fill = TRUE, header=TRUE)
# head(age_normal)
# age <- age_normal[age_normal$phase == '1기',]
dim(age)

[1] 2004   12

In [ ]:
p <- ggplot(age, aes(x=Horvath, group=phase, fill=phase)) + 
#   coord_cartesian(xlim = c(99, 100)) +
  geom_histogram(binwidth = 1, position='identity') + 
#   scale_x_continuous(breaks = seq(99, 100, 0.1)) +
  # scale_y_continuous(breaks = seq(90, 100, 2)) +
  labs(x='Age distribution', y='Estimated age by Horvath algorithm')
  theme_bw() +
  theme(
  #   axis.text.x = element_text(seq(0, 2198), angle=30, hjust=1),
    axis.text.x = element_text(size=12),
    axis.text.y = element_text(size=15),
    plot.title = element_text(hjust = 0.5,size=20,face='bold'),
    axis.title.x = element_text(size=15),
    axis.title.y = element_text(size=15),
    axis.ticks.x = element_blank(),
    legend.text = element_text(size=18),
    legend.position = 'none',
    
    panel.grid.major.x = element_line(linetype = "dotted", colour = "grey80"),
    panel.grid.major.y = element_line(linetype = "dotted", colour = "gray80"), 
    panel.grid.minor.x = element_blank(), 
    panel.grid.minor.y = element_blank(), 
    legend.title = element_blank()
    ) +
    scale_fill_manual(values=adjustcolor(c('#3A4D39', '#FF9209'),alpha.f = 0.5)) 

ggsave(p, file='/disk0/sm/methyl/03_pollution/total/plot/age_histo.png', width=6000, height=6000, units='px', dpi=1000)

In [ ]:
###### 그냥 양상 한번 봄 ## 아래는 실제 나이와 예측 나이를 비교한 것임

In [ ]:
age <- read.table('/disk0/sm/methyl/03_pollution/total/data/age_table.txt', sep='\t', header=T)
# age$rate = age$EN/age$Actual.age
head(age, 2)
# dim(age)

,Sample_Name,Age,Name,phase,Horvath,Hannum,Levine,skinHorvath,PedBE,Wu,BLUP,EN,SM
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,KoBB1-1317_T12_F3,41,KoBB1-1317,1기,54.87578,14.84841,35.22321,37.95929,14.56900,10.65192,46.76469,39.81933,42.43896
2,KoBB1-0525_T8_C4,48,KoBB1-0525,1기,60.76418,18.67244,47.67042,40.84970,13.33304,10.35417,52.50860,45.48900,47.44117


In [ ]:
algorithm <- names(age)[5:13]
for (i in algorithm){
  tmp <- age[c('Sample_Name', 'Age', i)]
  names(tmp) <- c('Sample_Name', 'Age', 'predict')
  print(paste(i, " : ", cor(tmp$Age, tmp$predict)))
  # print(paste(i, " : ", var(tmp$Age-tmp$predict)))
}

[1] "Horvath  :  26.392621098955"
[1] "Hannum  :  23.3825128432447"
[1] "Levine  :  31.9292633930556"
[1] "skinHorvath  :  12.678600968359"
[1] "PedBE  :  113.845917604781"
[1] "Wu  :  136.798715775958"
[1] "BLUP  :  8.63968427700684"
[1] "EN  :  9.28911313035061"
[1] "SM  :  2.35648565701509"


In [57]:
algorithm <- names(age)[5:13]
for (i in algorithm){
  tmp <- age[c('Sample_Name', 'Age', i)]
  names(tmp) <- c('Sample_Name', 'Age', 'predict')
  grob3 <- grobTree(textGrob(paste("Coefficient : ", cor(tmp$Age, tmp$predict)), x = 0.03, y = 0.97, hjust = 0, gp = gpar(col = "#ff4848", fontsize = 17, fontface = "bold")))
  p <- ggplot(data=tmp, aes(x=Age, y=predict)) +
  coord_cartesian(xlim = c(40, 90), ylim = c(20, 100))  +
  scale_x_continuous(breaks = seq(40, 90, 10)) +
  scale_y_continuous(breaks = seq(20, 100, 10)) +
  geom_point(size=0.5) +
    labs(title = i, 
          x = 'actual age', 
          y = 'predicted age') + 
    theme_bw() +
    theme(
    #   axis.text.x = element_text(seq(0, 2198), angle=30, hjust=1),
      axis.text.x = element_text(size=15),
      axis.text.y = element_text(size=15),
      plot.title = element_text(hjust = 0.5,size=20,face='bold'),
      axis.title.x = element_text(size=18),
      axis.title.y = element_text(size=18),
      axis.ticks.x = element_blank(), 
      legend.text = element_text(size=10),
      
      panel.grid.major.x = element_line(linetype = "dotted", colour = "grey80"),
      panel.grid.major.y = element_line(linetype = "dotted", colour = "gray70"), 
      panel.grid.minor.x = element_blank(), 
      panel.grid.minor.y = element_blank(), 
      legend.title = element_blank()
      ) +
      geom_smooth(method=lm, se=F, color='#ff0055') +
      annotation_custom(grob3)
  #   scale_colour_manual(values=c('#ff4040')) 
  # ggsave(p, file='/disk0/sm/chip/plot/final/total.png', width=2833, height=1890, units='px')
  ggsave(p, file=paste0('/disk0/sm/methyl/03_pollution/total/plot/age_', i, '.png'), width=8000, height=8000, units='px', dpi=1000)
}

`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'


In [ ]:
age_final <- read.table("/disk0/sm/methyl/03_pollution/total/data/age_algorithm.txt", sep="\t", fill = TRUE, header=TRUE)

# age_final <- age_final[order(age_final$Horvath), ]
# head(age_final)
# age_final$id <- factor(age_final$id, level=age_final$id)

m_age_final = melt(age_final)
head(m_age_final, 2)
# summary(age_final)
# m_age_final <- m_age_final %>% filter(variable != 'SM')

Using id as id variables



,id,variable,value
,<chr>,<fct>,<dbl>
1,KoBB1-1317_T12_F3,Horvath,13.87578
2,KoBB1-0525_T8_C4,Horvath,12.76418


In [ ]:
# m_age_final <- m_age_final %>% filter((variable == 'Horvath') | (variable == 'Levine') | (variable == 'BLUP') | (variable == 'SM'))

In [ ]:
p <- ggplot(m_age_final, aes(x=variable, y=value, fill=variable)) + 
  # coord_cartesian(ylim = c(-50, 50)) +
  geom_hline(yintercept=0, color='red', linetype = 2) +
  geom_boxplot(outlier.shape = NA) + #
  labs(
        x = 'age predict method', 
        y = 'age difference') + 
  # scale_fill_manual(values=c('#FB8072', '#FFED6F', '#ff6dbd', 'white')) +
  scale_fill_manual(values=c(colr, 'white')) +
  theme_bw() +
  theme(
    axis.text.x = element_text(size=15),
    axis.text.y = element_text(size=15),
    # plot.title = element_text(hjust = 0.5,size=10,face='bold'),
    axis.title.x = element_text(size=18),
    axis.title.y = element_text(size=18),
    axis.ticks.x = element_blank(), 
    legend.text = element_blank(), 
    
    panel.grid.major.x = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.major.y = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.minor.x = element_blank(), 
    panel.grid.minor.y = element_line(linetype = "dotted", colour = "grey70"),
    legend.title = element_blank(),
    legend.position = 'none',
  ) 
 
  # geom_jitter(shape=1, position=position_jitter(0.3), size=0.1) +
  #  geom_boxplot(width=0.1, outlier.shape = NA) 
  


  ggsave(p, file='/disk0/sm/methyl/03_pollution/total/plot/age_final.png', width=10000, height=10000, units='px', dpi=1000)
# dev.off()